In [1]:
import pandas as pd
from sqlalchemy import create_engine
from config import password

## Merge netflix and omdb csv files

In [2]:
file = "resources/netflix_titles.csv"
netflix_df = pd.read_csv(file)
file = "resources/omdb_list.csv"
omdb_df = pd.read_csv(file)
merged_df = netflix_df.merge(omdb_df, how="left", on="title")
merged_df

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,...,description,Unnamed: 0,genre,runtime,imdbRating,imdbVotes,poster,awards,boxoffice,language
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,...,Before planning an awesome wedding for his gra...,0.0,"Animation, Adventure, Comedy, Family",90 min,3.3,306,https://m.media-amazon.com/images/M/MV5BNjMwZD...,NaN,NaN,English
1,80117401,Movie,Jandino: Whatever it Takes,NaN,Jandino Asporaat,United Kingdom,"September 9, 2016",2016,TV-MA,94 min,...,Jandino Asporaat riffs on the challenges of ra...,1.0,Comedy,95 min,4.8,23,https://m.media-amazon.com/images/M/MV5BMWE3MG...,NaN,NaN,"English, Dutch"
2,70234439,TV Show,Transformers Prime,NaN,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,"September 8, 2018",2013,TV-Y7-FV,1 Season,...,"With the help of three human allies, the Autob...",2.0,"Animation, Action, Adventure, Comedy, Drama, F...",30 min,7.9,"5,454",https://m.media-amazon.com/images/M/MV5BMTczND...,14 wins & 26 nominations.,NaN,English
3,80058654,TV Show,Transformers: Robots in Disguise,NaN,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,"September 8, 2018",2016,TV-Y7,1 Season,...,When a prison ship crash unleashes hundreds of...,3.0,"Animation, Action, Adventure, Comedy, Sci-Fi",22 min,6.0,842,https://m.media-amazon.com/images/M/MV5BMjMwNT...,2 wins & 11 nominations.,NaN,English
4,80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,"September 8, 2017",2017,TV-14,99 min,...,When nerdy high schooler Dani finally attracts...,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6229,80000063,TV Show,Red vs. Blue,NaN,"Burnie Burns, Jason Saldaña, Gustavo Sorola, G...",United States,NaN,2015,NR,13 Seasons,...,"This parody of first-person shooter games, mil...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6230,70286564,TV Show,Maron,NaN,"Marc Maron, Judd Hirsch, Josh Brener, Nora Zeh...",United States,NaN,2016,TV-MA,4 Seasons,...,"Marc Maron stars as Marc Maron, who interviews...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6231,80116008,Movie,Little Baby Bum: Nursery Rhyme Friends,NaN,NaN,NaN,NaN,2016,NaN,60 min,...,Nursery rhymes and original music for children...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6232,70281022,TV Show,A Young Doctor's Notebook and Other Stories,NaN,"Daniel Radcliffe, Jon Hamm, Adam Godley, Chris...",United Kingdom,NaN,2013,TV-MA,2 Seasons,...,"Set during the Russian Revolution, this comic ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Transform netflix genre

In [3]:
# Turn listed_in into list
merged_df.listed_in = merged_df.listed_in.str.split(',').tolist()
listed_df = merged_df.dropna(subset=['listed_in'])
listed_in = listed_df["listed_in"].tolist()

In [4]:
# Loop through listed_in (netflix genre) and get unique values into netflix_genre set
netflix_genre = set()

for movie in listed_in:
    for x in movie:
        netflix_genre.add(x)
print(netflix_genre)

netflix_genre_id = pd.DataFrame(netflix_genre, columns=["netflix_genre"])
netflix_genre_id = netflix_genre_id.reset_index()
netflix_genre_id = netflix_genre_id.rename(columns={'index': 'netflix_genre_no'})
netflix_genre_id.head()

{'Documentaries', "Kids' TV", ' LGBTQ Movies', 'Romantic Movies', 'Sci-Fi & Fantasy', 'Horror Movies', ' Sci-Fi & Fantasy', 'Reality TV', ' Korean TV Shows', 'Stand-Up Comedy & Talk Shows', ' TV Action & Adventure', ' Comedies', ' TV Thrillers', ' Faith & Spirituality', 'Anime Series', ' Romantic TV Shows', ' Spanish-Language TV Shows', ' Science & Nature TV', 'Thrillers', ' Children & Family Movies', 'Classic & Cult TV', 'Classic Movies', 'TV Sci-Fi & Fantasy', ' Horror Movies', 'Children & Family Movies', 'International TV Shows', 'Stand-Up Comedy', 'Anime Features', ' TV Horror', 'TV Action & Adventure', ' Documentaries', ' Classic Movies', 'Dramas', 'Sports Movies', ' Romantic Movies', ' Dramas', ' Reality TV', ' Independent Movies', ' International TV Shows', 'Crime TV Shows', 'Cult Movies', 'TV Comedies', 'Movies', 'Action & Adventure', ' Sports Movies', 'Independent Movies', ' TV Sci-Fi & Fantasy', ' TV Mysteries', 'Comedies', 'Music & Musicals', 'TV Horror', ' TV Dramas', 'TV D

,netflix_genre_no,netflix_genre
0,0,Documentaries
1,1,Kids' TV
2,2,LGBTQ Movies
3,3,Romantic Movies
4,4,Sci-Fi & Fantasy


In [5]:
# Break out genres (listed_in) with show_id
listed_in_df = merged_df.loc[:,['show_id', 'listed_in']]
listed_in_df = listed_in_df.explode('listed_in')
listed_in_df = listed_in_df.rename(columns={'listed_in': 'netflix_genre'})
listed_in_df.head()

,show_id,netflix_genre
0,81145628,Children & Family Movies
0,81145628,Comedies
1,80117401,Stand-Up Comedy
2,70234439,Kids' TV
3,80058654,Kids' TV


In [6]:
# merge tables to create show_id and genre_id table
netflix_genre_table = listed_in_df.merge(netflix_genre_id,how = "left", on="netflix_genre")
netflix_genre_table = netflix_genre_table.loc[:,['show_id','netflix_genre_no']]
netflix_genre_table.head()                                        

,show_id,netflix_genre_no
0,81145628,24
1,81145628,11
2,80117401,26
3,70234439,1
4,80058654,1


## Transform omdb genre

In [7]:
# Turn genre into list 
merged_df.genre = merged_df.genre.str.split(',').tolist()
genre_df = merged_df.dropna(subset=['genre'])
genre = genre_df["genre"].tolist()

In [8]:
# Loop through genre (omdb genre) and get unique values into omdb_genre set
omdb_genre = set()

for movie in genre:
    for y in movie:
        omdb_genre.add(y)
print(omdb_genre)

omdb_genre_id = pd.DataFrame(omdb_genre, columns=["omdb_genre"])
omdb_genre_id = omdb_genre_id.reset_index()
omdb_genre_id = omdb_genre_id.rename(columns={'index': 'omdb_genre_no'})
omdb_genre_id.head()

{' Sci-Fi', 'Crime', 'Horror', ' Crime', ' Action', ' Comedy', 'Comedy', ' Musical', 'Fantasy', 'Short', ' Fantasy', ' Talk-Show', ' Music', ' Drama', ' News', ' Thriller', 'Documentary', ' Short', ' War', ' Family', 'Adventure', 'Animation', 'Biography', 'History', ' Adventure', 'Drama', ' Mystery', 'Mystery', 'Action', 'Reality-TV', ' Biography', 'Thriller', ' Horror', ' Sport', ' History', ' Romance'}


,omdb_genre_no,omdb_genre
0,0,Sci-Fi
1,1,Crime
2,2,Horror
3,3,Crime
4,4,Action


In [9]:
# Break out genres with show_id
genre_df = merged_df.loc[:,['show_id', 'genre']]
genre_df = genre_df.explode('genre')
genre_df = genre_df.rename(columns={'genre': 'omdb_genre'})
genre_df.dropna(inplace = True)
genre_df.head()

,show_id,omdb_genre
0,81145628,Animation
0,81145628,Adventure
0,81145628,Comedy
0,81145628,Family
1,80117401,Comedy


In [10]:
# merge tables to create show_id and genre_id table
omdb_genre_table = genre_df.merge(omdb_genre_id,how = "left", on="omdb_genre")
omdb_genre_table = omdb_genre_table.loc[:,['show_id','omdb_genre_no']]
omdb_genre_table.head()

,show_id,omdb_genre_no
0,81145628,21
1,81145628,24
2,81145628,5
3,81145628,19
4,80117401,6


## Transform language table

In [11]:
# Turn language into list 
merged_df.language = merged_df.language.str.split(',').tolist()
lan_df = merged_df.dropna(subset=['language'])
language = lan_df["language"].tolist()
language

[['English'],
 ['English', ' Dutch'],
 ['English'],
 ['English'],
 ['French', ' Arabic'],
 ['English'],
 ['Spanish'],
 ['English'],
 ['English'],
 ['English', ' Dutch', ' German'],
 ['Hindi', ' English'],
 ['English', ' French'],
 ['English'],
 ['English'],
 ['Spanish', ' English'],
 ['English'],
 ['English'],
 ['English'],
 ['English'],
 ['English', ' Mandarin'],
 ['English'],
 ['French'],
 ['French'],
 ['Hindi'],
 ['Telugu'],
 ['English', ' Arabic'],
 ['English'],
 ['Hindi'],
 ['English'],
 ['English'],
 ['English'],
 ['English'],
 ['Thai'],
 ['Thai'],
 ['English'],
 ['Thai'],
 ['Thai'],
 ['Thai'],
 ['English'],
 ['English'],
 ['English'],
 ['Urdu'],
 ['English'],
 ['English'],
 ['Tamil'],
 ['English', ' Swahili', ' Nama', ' Xhosa', ' Korean'],
 ['English'],
 ['English'],
 ['English'],
 ['English'],
 ['English'],
 ['English'],
 ['English'],
 ['English'],
 ['English'],
 ['English'],
 ['English'],
 ['English'],
 ['English'],
 ['English', ' Italian'],
 ['English'],
 ['English'],
 ['Engl

In [12]:
# Loop through language (omdb language) and get unique values into language_set set
language_set = set()

for row in language:
    for i in row:
        language_set.add(i)
print(language_set)

language_df = pd.DataFrame(language_set, columns=["language"])
language_df = language_df.reset_index()
language_df = language_df.rename(columns={'index': 'language_no'})
language_df.head()

{' English', ' Yoruba', 'Portuguese', 'Tamil', 'Urdu', 'Filipino', ' Tagalog', ' Xhosa', 'Dutch', 'German', ' Dutch', ' Minangkabau', 'Spanish', 'Turkish', ' Ibo', ' Italian', 'Japanese', ' Swahili', ' Indonesian', 'American Sign Language', 'Indonesian', ' Hebrew', ' Japanese', ' Serbo-Croatian', ' Cantonese', 'Vietnamese', 'English', 'Hindi', ' French', ' Arabic', ' Catalan', ' Mandarin', ' Hausa', ' Russian', ' Nama', ' Vietnamese', ' Spanish', ' Swedish', 'Thai', 'Yoruba', 'French', 'Telugu', 'Mandarin', 'Catalan', 'Arabic', ' Latin', ' German', 'Cantonese', ' Korean', 'Korean', 'Chinese'}


,language_no,language
0,0,English
1,1,Yoruba
2,2,Portuguese
3,3,Tamil
4,4,Urdu


In [13]:
# Break out languages with show_id
language_all_df = merged_df.loc[:,['show_id', 'language']]
language_all_df = language_all_df.explode('language')
language_all_df.dropna(inplace = True)
language_all_df.head()

,show_id,language
0,81145628,English
1,80117401,English
1,80117401,Dutch
2,70234439,English
3,80058654,English


In [14]:
# merge tables to create show_id and language_no table
language_table = language_all_df.merge(language_df,how = "left", on="language")
language_table = language_table.loc[:,['show_id','language_no']]
language_table.head()

,show_id,language_no
0,81145628,26
1,80117401,26
2,80117401,10
3,70234439,26
4,80058654,26


## Load data into postgres using sqlalchemy

In [ ]:
# create connection to ETL_project_DB in postgres
engine = create_engine('postgresql://postgres:'+ password + '@localhost:5432/ETL_project_DB')
connection = engine.connect()

In [ ]:
# check table names in ETL_project_DB
engine.table_names()

In [ ]:
# load dataframes into postgres using pandas
merged_df.to_sql(name='Title', con=engine, if_exists='append', index=False)
netflix_genre_table.to_sql(name='Netflix_title_Listed_in', con=engine, if_exists='append', index=False)
netflix_genre_id.to_sql(name='Netflix_Listed_in', con=engine, if_exists='append', index=False)
omdb_genre_table.to_sql(name='OMDB_title_genre', con=engine, if_exists='append', index=False)
omdb_genre_id.to_sql(name='OMDB_genre', con=engine, if_exists='append', index=False)
language_table.to_sql(name='OMDB_title_language', con=engine, if_exists='append', index=False)
language_df.to_sql(name='OMDB_language', con=engine, if_exists='append', index=False)